In [1]:
# from transformers import BertTokenizer, BertModel
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, BucketIterator, TabularDataset

import spacy
import numpy as np

import random
import math
import time
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
"/content/drive/MyDrive/DLNLP project/socialnetwork.paraphrases.train.examples"
"/content/drive/MyDrive/DLNLP project/socialnetwork.paraphrases.test.examples"

with open('/content/drive/MyDrive/DLNLP project/publications.paraphrases.train.examples','r') as f:
  examples = f.readlines()
  lines = [e.strip().split('\n') for e in examples]


utterance = []
original = []

for line in lines:
  if line[0].find('(utterance') == 0:
    utterance.append(line[0][12:-2])
  if line[0].find('(original') == 0:
    original.append(line[0][11:-2])


import pandas as pd

df = pd.DataFrame({'utterance':utterance, 'original':original})

df['utterance'] = df['utterance'].apply(lambda x : ' '.join(x.split()))
df['original'] = df['original'].apply(lambda x : ' '.join(x.split()))
df.to_csv('train_data.csv',index=False)

with open('/content/drive/MyDrive/DLNLP project/publications.paraphrases.test.examples','r') as f:
  examples = f.readlines()
  lines = [e.strip().split('\n') for e in examples]


utterance = []
original = []

for line in lines:
  if line[0].find('(utterance') == 0:
    utterance.append(line[0][12:-2])
  if line[0].find('(original') == 0:
    original.append(line[0][11:-2])

df = pd.DataFrame({'utterance':utterance, 'original':original})

df['utterance'] = df['utterance'].apply(lambda x : ' '.join(x.split()))
df['original'] = df['original'].apply(lambda x : ' '.join(x.split()))
df.to_csv('test_data.csv',index=False)

In [5]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [6]:
train_df.tail()

,utterance,original
635,what article cites the fewest articles,article that cites the least number of article
636,what is an article that does not cite multivar...,article that multivariate data analysis not cites
637,find an article with no more than two venues,article that has at most two venue
638,name an article found in two venues,article that has two venue
639,what 2004 article was cited by multivariate da...,article whose publication date is 2004 and tha...


CD cardinal digit                                                               
DT determiner                                                             
JJ adjective ‘big’                               
JJR adjective 'bigger' comparative                                                                                                     
JJS adjective, superlative   'biggest'                                                        
NN noun, singular ‘desk’                                                                                                
NNS noun plural ‘desks’                                                                                  
NNP proper noun, singular ‘Harrison’                                                            
NNPS proper noun, plural‘Americans’                                                                                                 
RB adverb very, silently,                             
RBR adverb, comparative 'better'                                                                        
RBS adverb, superlative 'best'

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import PunktSentenceTokenizer
stop_words = set(stopwords.words('english'))
  

sentence_tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

dict_NP_count_chunk = {}

grammar = r"NP: {<DT>?<JJ.*|RB.*|CD>*<NN.*>+}"
# grammar = r"""NP: {<DT|PP\$>?<JJ>*<NN>}{<NNP>+}"""

Parser = nltk.RegexpParser(grammar)
unique_NP = []
NP_list = []

for sentence in train_df['original'].tolist() +test_df['original'].tolist():
    
    wordsList = sentence_tokenizer.tokenize(sentence)[0].split()

    tagged = nltk.pos_tag(wordsList)
    # print(tagged)
    pos_list = str(Parser.parse(tagged)).split()
    # print(pos_list)

    # count = 0
    temp_list = []
    temp_set =  set()

    while True:
      try:
        id = pos_list.index('(NP')
        # count += 1
        tok = []
        for dummy in pos_list[id+1:]:
          dummy = dummy.split('/')
          tok.append(dummy[0])

          if len(dummy[1]) >= 3 and dummy[1][-1] == ')':
            break

        pos_list = pos_list[id+1:]
        unique_NP.append(' '.join(tok))

        if ' '.join(tok) not in temp_set:
          temp_list.append(' '.join(tok))
          temp_set.add(' '.join(tok))
      except:
        break

    try:
      dict_NP_count_chunk[len(temp_list)].append((sentence,temp_list))
    except:
      dict_NP_count_chunk[len(temp_list)] = []
      dict_NP_count_chunk[len(temp_list)].append((sentence,temp_list))

    NP_list.append(temp_list)

# print(dict_NP_count_chunk)
unique_NP = set(unique_NP)
print(unique_NP)
# train_df['NP_list'] = NP_list

{'data analysis', 'publication date', 'number', 'most two article cites', 'computational linguistics', 'venue', 'multivariate data analysis', 'author', 'two author', 'two venue', 'an award cites', 'least publication date', 'most publication date', 'least two article cites', 'the most number', '2004 cites', 'efron cites', 'least two article', 'article', 'two article cites', 'least two author', 'cites', 'that multivariate data analysis cites', 'most two venue', 'person', 'multivariate data analysis cites', 'an award', 'the least number', 'article cites', 'least two venue', 'the smallest publication date', 'the largest publication date', 'statistics cites', 'annals', 'most two author', 'most two article', 'two article', 'statistics'}


In [8]:
train_df.head()

,utterance,original
0,article with the largest amount of authors,article that has the most number of author
1,article citing article published in annals of ...,article that article whose venue is annals of ...
2,what article from 2004 cites multivariate data...,article whose publication date is 2004 and tha...
3,find an article published in 2004,article whose publication date is 2004
4,articles that have a publication date close to...,article whose publication date is at most publ...


In [9]:
print(f"max number of Noun phrases a sentence has : {len(dict_NP_count_chunk.items())}")
print(f"Length of noun phrases : {dict_NP_count_chunk.keys()}")
print(f"#Unique Noun phrases in the Dataset : {len(unique_NP)}")

max number of Noun phrases a sentence has : 6
Length of noun phrases : dict_keys([3, 4, 2, 5, 6, 0])
#Unique Noun phrases in the Dataset : 38


In [10]:
# dict_NP_count_chunk[1]

In [11]:
test_df.head()

,utterance,original
0,what article has the most number of articles c...,article that the most number of article cites
1,person who has not published article in multiv...,person that is not author of multivariate data...
2,what person is not the author of multivariate ...,person that is not author of multivariate data...
3,article cited by two articles,article that two article cites
4,articles that do not cite multivariate data an...,article that not cites multivariate data analysis


In [12]:
test_NP_list = []

for sentence in test_df['utterance']:
    
    wordsList = sentence_tokenizer.tokenize(sentence)[0].split()

    tagged = nltk.pos_tag(wordsList)
    # print(tagged)
    pos_list = str(Parser.parse(tagged)).split()
    # print(pos_list)

    # count = 0
    temp_list = []
    temp_set = set()

    while True:
      try:
        id = pos_list.index('(NP')
        # count += 1
        tok = []
        for dummy in pos_list[id+1:]:
          dummy = dummy.split('/')
          tok.append(dummy[0])
          if len(dummy[1]) >= 3 and dummy[1][-1] == ')':
            break
        pos_list = pos_list[id+1:]
        
        if ' '.join(tok) not in temp_set:
          temp_list.append(' '.join(tok))
          temp_set.add(' '.join(tok))
      except:
        break

    test_NP_list.append(temp_list)


In [13]:
test_df['NP_utterance_list'] = test_NP_list
test_df.head()

,utterance,original,NP_utterance_list
0,what article has the most number of articles c...,article that the most number of article cites,"[article, the most number, articles]"
1,person who has not published article in multiv...,person that is not author of multivariate data...,"[person, article, multivariate data analysis]"
2,what person is not the author of multivariate ...,person that is not author of multivariate data...,"[person, the author, multivariate data analysis]"
3,article cited by two articles,article that two article cites,"[article, two articles]"
4,articles that do not cite multivariate data an...,article that not cites multivariate data analysis,"[articles, multivariate data analysis]"


In [14]:
exact_match = []

for  NP_list,target in zip(test_df['NP_utterance_list'].tolist(),test_df['original'].tolist()):


  # check alignment
  max_similarity = -1
  best_mapping = ""
  # similarity_score = -1

  if len(NP_list) not in dict_NP_count_chunk.keys():
    continue

  for sentence in dict_NP_count_chunk[len(NP_list)]:
    similarity_score = 0
    # print(NP_list,sentence[1])

    assert len(NP_list) == len(sentence[1])

    for x,y in zip(NP_list,sentence[1]):
      # print(f"{x},{y}")
      similarity_score += int(x == y)
    
      if max_similarity < similarity_score:
        best_mapping,max_similarity = sentence[0],similarity_score
  
  if target == best_mapping:
    exact_match.append(f"{best_mapping} ## {target} ## {max_similarity}")

print(exact_match[:5])

['article that has the smallest publication date ## article that has the smallest publication date ## 2', 'article that has at most two author ## article that has at most two author ## 2', 'venue that is venue of less than two article ## venue that is venue of less than two article ## 1', 'article that cites the most number of article ## article that cites the most number of article ## 2', 'venue that is venue of less than two article ## venue that is venue of less than two article ## 2']


In [22]:
print(len(exact_match))

48


In [16]:
print(len(test_df))

161
